# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
  # Scroll down to the bottom.  self.
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 踩到金管會紅線！　街口「託付寶」有規避監理問題

▲金管會。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
「街口支付 託付寶」廣告打出透過街口的電子支付帳戶買基金，保證年收益1.5%，再度踩到金管會紅線，認為有規避監理的問題；街口投信今（3）日澄清，內容提及街口投信及該公司基金，均未經本公司審閱與同意，與本公司業務無涉。
街口支付在2018年曾預告推出「街口託付帳戶」，號稱為台版餘額寶，並在宣傳中保障年收益1.2%至1.8%，不過卻踩到金管會紅線，要求要調整行銷內容，如今再度刊出「街口支付 託付寶」廣告，打出保證年收益1.5%，再度引發各界關注。
街口投信發出聲明指出，街口支付所發布的「街口支付 託付寶」相關廣告，內容提及街口投信及該公司基金，均未經本公司審閱與同意，與本公司業務無涉，將恪守投信投顧相關從業規範，並善盡善良管理人的義務與職責。
至於保證收益一事，街口投信澄清，以往績效不保證基金的最低投資收益，強調公司基金都經過金管會核准或申報生效，但不表示絕無風險；除盡善良管理人的注意義務外，不負責基金的盈虧，也不保證最低收益，投資人申購前應詳閱基金公開說明書。
證期局官員表示，因應銀行局修正民眾可透過電子支付購買投信發行的新台幣計價基金相關規定，證期局也配合修正投信投顧公會自律規範，若未來想要以這種方式買基金，就必須要修改合約後，才能與電支機構合作。
證期局官員指出，電支平台不可做涉及基金銷售業務及廣告，因此投信若要與電支平台合作，必須要先核閱相關規範；至於對該廣告是否進行處分，官員則說，只能管到街口投信，而託付寶的母公司是街口金融科技公司。
----------------------------------------------------------------------

[1] 影／海外投資房產不容易　房產達人：東南亞需特別留意

台灣房產投資報酬率低、房租收益率也低，因此，不少投資人開始注意到海外房產投資。但是在《雲端最有錢》節目中，Domo民宿執行長李奇悅表示，如果海外房產投報率沒有台灣的一半的話，建議不要去，因為海外置產還要扣掉很多成本，像是稅費、物業管理費等等。
李奇悅提到，房地產不像股票一樣，

▲ 美股2020年仍受看好。（圖／路透社） 
記者李瑞瑾／綜合報導
統計顯示，當標普500股利率大於美債10年期殖利率時，任意時間進場投資美股1年，平均年化報酬為16%，高於所有時間的平均年化報酬10%。
選舉年前漲得多、選舉後波動大
國泰投顧提醒，2020年美國股市最大風險是11月總統大選。儘管川普於2016年上任以來主打貿易戰，造成市場波動，但是川普的支持度並未明顯下降，加上美國總統競選連任素有優勢，歷史上少見連任失敗案例，估計川普在2020年11月勝出的機率頗高。
國泰投顧認為，川普連任對市場的影響可能最小，主要因為投資人已經熟悉川普的言行習性。另外，民主黨候選人當中民調領先者拜登、華倫與桑德斯屬改革派，主打的議題包括：科技監管、富人稅、平價醫療…等，可能在未來一年的競選活動中影響市場情緒。統計過去10次總統大選的美國股市表現，除2008年之外，可以發現選舉年的上半年股市漲得多，而越接近選舉日，不管是選舉前或後，股市波動都明顯較大。
----------------------------------------------------------------------

[11] 終身主約保險「較貴又沒功能」！保單健檢先砍殺大問題…專家3招實用技巧曝光

▲年終期間，民眾應全面檢視自己的保障結構，但到底該如何了解保單情形呢？卻也是多數人的煩惱。（圖／pakutaso）
記者楊絡懸／台北報導
邁入新的一年，許多事物都要除舊布新。雖然常聽說要「年度保單健檢」，但民眾該如何自行檢查全家保障的保單呢？錠嵂保險經紀人教學3個簡單的技巧，靠自己也可以檢查保單是否有問題或保障需求不夠。
步驟一：思考自身買保險的目的與需求
錠嵂保經表示，每個人買保險的目的不同，要解決的問題也不一樣。例如，若是高資產族群，可能會著重在理財與投資效果，這時考量除了財產能否順利節稅或傳承外，就是獲利指數了，可用「內部報酬率」（IRR）作為參考指標。
▲容易造成經濟壓力的保單。（圖／記者楊絡懸製表）
步驟二：先找大問題！有無過高又沒必要的終身主約？
錠嵂保經指出，保單組合通常由少數幾張「終身主約」，搭配許多帶有重點保障內容的「定期附約」構成。當經濟能力沒特別高的情況下，卻發現組合裡居然有終身型健康保險，就要特別注意。
步驟三：檢查小地方！各定期附約保障是否足夠且划算？
▲「六大險種保障」附約檢查

▲榮町投資公司總監蔡泳瀠（左2），成功挽回老招牌榮光的故事。（圖／商業周刊／攝影程思迪）
文／陳承璋
全買，曾經是雲嘉南最大超市，最高峰年營收可達40億元，2014年，因為受黑心油風暴衝擊，僅半年時間就宣告熄燈；3年之後，熄燈的超市，竟自創新品牌，現在更成為了第一個把本土黃豆推到大型通路的廠商。
這個品牌是榮町，而創辦人是它的二代，蔡泳瀠。這是一個超市二代，要挽回老招牌榮光的故事。
去年，11月中旬，統一超上市了一瓶採用台灣本土黃豆的豆奶紅茶，這是本土豆近6年來復耕風潮興起後，第一次大規模上架大型通路。上架第一週，即賣破10萬罐，衝至統一超傳統飲料類別的前3名，與統一、愛之味等食品廠並列。
榮町是一間成立3年的嘉義地方創生農產品牌，1年營收僅千萬規模，又新、又小，竟能打進大通路，打破很多農業界人士的眼鏡。
因為這個困難度，甚至連義美都辦不到。
2016年，義美對外宣布，為了支持台灣本土黃豆，以貴進口黃豆2倍的價錢收購，做成小包裝豆漿，但也僅有在自家門市與少數通路小量鋪貨，台灣雜糧發展基金會代理執行長焦鈞說，義美只有小量生產，主要在農會這種封閉型通路賣，關鍵還是台灣本土黃豆成本太高，量產後產品價格上一般通路，會很難推。
1993年成立的全買，在嘉義靠著低價產品策略，花20年成為1年營收超過40億元的雲嘉南超市龍頭，本來在台北開廣告動畫公司的蔡泳瀠，2012年被父親蔡榮昌找回全買，成立網購新事業。
2013年，全台陷入黑心油風暴中，採取低價策略的全買，採購的產品幾乎都以低價為主，這讓全買高達6成的商品，都向爆發黑心油的廠商進貨。全買的營收受黑心油牽連甚深，竟衰退超過一半，因不堪打擊，只好宣布把大店鋪賣給全聯，小店鋪由美廉社瓜分，就此熄掉燈號。
這是她與父親一輩子的痛。她甚至自責：「我是一個失敗的二代，如果我提早回去，可能可以提早讓全買改革，提早不再用低價去競爭，也許損失就不會這麼大，全買還會在…。」
沉潛2年時間，先是說服全買老股東，從過去的痛出發，成立榮町投資公司，在全買原總部地址蓋榮町市場，專門賣小農產品。她也說服爸爸同意共同出資2千萬。
她說，當時急於想走出食安陰影，重新在嘉義站起來，一股腦去做，卻忘了，小農市集因農人有機小量耕種，價格都很高。其次，小農供貨，容易因天候斷貨，品質落差很大，有些送到市集就已爛掉，導致市集撐了一年半，獲利不佳，宣布收掉，2千萬元如同放

▲伊朗少將蘇萊曼尼（Qasem Soleimani）被證實在空襲中喪生。（圖／達志影像／美聯社）
記者王曉敏／綜合報導
美國國防部證實，伊朗少將、菁英部隊「聖城軍」（Quds Force）指揮官蘇萊曼尼（Qassem Soleimani）已於美軍空襲中喪身。消息一出，油價、金價應聲飆漲。
綜合外電報導，美國總統川普下令空襲伊拉克的巴格達國際機場，五角大廈在聲明中表示，美軍採取了「果斷的防禦行動，以保護在海外的美國人員。」而伊朗少將蘇萊曼尼則在這場空襲中罹難，「人民動員」的副指揮官穆罕迪斯（Abu Mahdi al-Muhandis）也不幸喪生。
消息一出，倫敦布蘭特原油及紐約西德州中級原油皆一度上漲逾4%，達到去年9月沙烏地阿拉伯石油設施遭無人機攻擊以來的最高單日漲幅。《彭博》指出，雖然此次事件並無任何石油設備或生產受到影響，但擊斃伊朗革命衛隊指揮官無疑是種挑釁，可能加劇華盛頓與伊斯蘭國家之間的衝突。
AxiTrader首席市場策略師英尼斯（Stephen Innes）在報告中表示，這是激進的武力表現和徹底的挑釁，可能會引發另一場中東戰爭。
截至新加坡午盤，原油價格回吐了部分近期漲幅，但仍處於9月份以來新高。倫敦布蘭特原油3月交割價上漲2.9%至每桶68.20美元，此前其一度上漲4.4％至每桶69.16美元。紐約西德州中級原油2月期貨價格上漲2.7%至每桶62.83美元，今日也一度上漲4.4％至63.84美元，超越去年9月的價格，創自去年5月以來新高。
與此同時，金價也攀升至4個月來高點。Phillip Futures分析師盧（Benjamin Lu）表示，中東的消息，加上一些技術性買盤及美元疲軟，支撐了黃金價格。
截至今日上午10點50分，黃金現貨價格上漲0.6％觸及9月5日以來新高，達1540.48美元；黃金期貨則上漲0.9％，報每英兩1541.20美元。在政治和經濟不確定時，黃金被認為是安全的投資，總計本周黃金現貨已上漲1.9%，已為連續第4周上漲。
----------------------------------------------------------------------

[33] 英鎊看漲至1.65美元！策略師：英鎊將成最大交易量幣別之一

▲策略師看好英鎊今年升值。（圖／達志影像／美聯社）
記者謝仁傑／綜合報導
投資機構Cribston

▲ 日圓匯價又開始走高。（圖／Pixabay）
記者李瑞瑾／台北報導
日圓匯價前兩周維持在0.27震盪，但近期又開始走高。根據台銀今（3）日上午牌告的日圓現金賣出匯價，日圓來到0.2814元，創下去年12月中旬以來新高。
綜合外媒報導，法國農業信貸銀行的外匯策略師在報告中強調了美元相對日元的弱勢，他們指出，這對貨幣在過去10年中有7年下跌；東京農業信貸銀行外匯部門的執行董事也表示：「自2014年1月以來，美元兌日元僅上漲了一次。考慮到這一點，交易者可能希望在2019年年底或2020年年初左右賣出美元。」
另一方面，由於美國與伊朗之間的緊張關係加劇，國際資金避險情緒升溫，也讓日圓兌美元匯率走高。
雖然日本依舊在新年假期中，但日圓兌美元匯率還是不斷走高。目前為108.19日圓兌1美元，創下去年11月1日（匯率同為108.19日圓兌1美元）、兩個月以來新高。
另外，根據台銀今日上午牌告的日圓現金賣出匯價，日圓來到0.2814元，為12月12日匯價0.2829元以來新高價。
哈日族若拿新台幣5萬元換日圓，今天可以換到17萬7,638日圓；相較去年12月19日的低點匯價0.2787，當時可以換到17萬9,404日圓，荷包足足縮水了1,766日圓，約等於少吃了兩碗拉麵。
匯率隨時變動，如果哈日族近期要赴日遊玩，或是想換日元備著用的話，可以把握低點分批換起來，以免買在高點的時候傷了荷包。
----------------------------------------------------------------------

[45] 「股東e票通」資安再升級　電子投票多一項驗證防惡意干擾

▲ 電子投票只要用手機或電腦就能完成，不出席股東會也可以行使表決權。（示意圖／pixabay）
記者李瑞瑾／台北報導
邁入109年，集保結算所「股東e票通」電子投票平台服務，可協助投資人以快速便捷及多元化的電子化途徑行使股東表決權，程式更在1月起引入「我不是機器人」驗證，防止惡意程式干擾投資人電子投票。
集保結算所表示，108年採用電子投票上市（櫃）、興櫃公司股東常會共1,757家，電子投票占股東會出席股數比率52.54%，此外投票總筆數超過737萬筆，行使表決總權數亦高達3,133億餘股，電子投票不論在總筆數、權數，甚至投票比率均再創新高，可見股東行動主義的推展成效。
網路時代的資通安

▲南山人壽總公司。（圖／記者湯興漢攝）
記者楊絡懸／台北報導
南山人壽原擬定潤泰集團總裁尹衍樑獨子尹崇堯代理董事長，在金管會否決人事案後，南山人壽宣布尹崇堯請辭代理董事長，改由南山產險前董事長陳棠代理。不過，內部公告卻流出，當天董事會還決議通過尹崇堯出任副董事長一職。
根據南山人壽2019年12月13日當天的內部公告，董事會決議通過設立副董事長一人，並由全體董事推舉尹崇堯為副董事長，同時也通過副董事長職權可獲董事長授權，督導南山人壽的業務執行與處理內外事宜，並指派兼任公平待客推展委員會召集人及投資審議委員會主委與主席。
針對尹崇堯出任南山人壽副董事長一事，南山工會表示不知情，內部業務員也是方才得知。不過，南山工會理事長蔡坤穎表示，無論是陳棠或尹崇堯出任管理職，應儘快處理南山人壽新系統及勞資糾紛，改善南山同仁的權益而能安心過年。
南山人壽因「境界」新系統自2018年9月以來錯誤百出，影響多數保戶權益，去年9月遭金管會重罰3000萬元，時任董事長杜英宗也遭停職2年處分，停職期間不得支付任何報酬，同時也暫停販售投資型保單，直至系統完全改善為止，是保險公司史上金額最大筆的罰單。
在金管會「兩次面試」尹崇堯，綜合評估後，保險局長施瓊華宣布代理董座尹崇堯人事案「緩議」，原因是考量南山人壽資產及業務規模龐大，就擔任代理董座一職而言，尹崇堯在保險專業能力、保險業經營經驗方面，仍有待進一步歷練提升及累積經驗，「不是年紀輕的緣故，而是須具備一定資歷。」
----------------------------------------------------------------------

[60] 麗豐-KY布局再生醫學　上海成立「同濟大學麗豐再生醫學研究院」

▲麗豐攜手同濟大學共同打造「同濟大學麗豐再生醫學研究院」。（圖／麗豐提供）
記者姚惠茹／台北報導
麗豐-KY（4137）今（2）日宣布在上海攜手同濟大學共同成立「同濟大學麗豐再生醫學研究院」，打造GMP規格的全新的研究實驗中心，由麗豐-KY董事長陳碧華與同濟大學醫學院教授張軍進行揭牌落成啟動，並宣布由陳碧華擔任研究院院長一職。
麗豐-KY表示，同濟大學在幹細胞研究和再生醫學研發中具有領先的學術研發地位，又以具有自我複製能力和多向分化潛能的「間質幹細胞(mesenchymal stem cell, MSC) ｣相關轉化研究

看好網路、保險、媒體、國防航太、製藥、石油等產業的表現，點名Alphabet、波克夏Ａ股、Comcast、ViacomCBS、戴爾、聯合科技和殼牌石油等股票。
文／魏聖峰
歲末年初，霸榮周刊都會針對新年度挑選十檔看好的股票。二○一九年S&P 500指數累積上漲二八．五％，創下二○一三年以來最好的年度表現，這波美股的牛市已經延續十一年。
霸榮周刊依照股票價值、未來的獲利展望和本益比等三個條件選股。二○二○年看好的十檔股票包括，波克夏Ａ股、媒體股Comcast和ViacomCBS、石油股皇家殼牌石油、製藥業的輝瑞大藥廠、管理式醫療的安森、電腦硬體的戴爾、搬家工具出租U-Haul母公司的Amerco、國防航太的聯合科技以及網路股Alphabet。其中，Alphabet是連續兩年被挑選中。
根據霸榮周刊的資料顯示，二○二○年看好的十檔股票平均本益比約為十四倍，股息殖利率為一．六四％。相比之下，S&P 500指數成分股的平均本益比為十九．七一倍，股息殖利率為一．八％。
一年前霸榮周刊挑選一九年度看好的十檔股票中，表現最好的是蘋果，股價漲幅高達八成、美國銀行漲幅也有近四三％，是表現最好的兩檔股票。能源轉換（Energy Transfer）下跌一．○六％是唯一下跌的股票。這十檔股票的平均報酬率為二四．八八％，略低於S&P 500指數的上漲二八．五％表現。
Alphabet連續兩年被霸榮周刊選中，身為谷歌的控股公司，在一九年股價漲幅接近三成。若以二○二○年預估ＥＰＳ的二五美元換算成本益比是二五倍。不過，谷歌表示財報打消沒有商業化部分虧損，例如自動車Waymo後，使得ＥＰＳ約二○美元本益比提高到二八．六六倍。
Alphabet財務體質堅強
Alphabet的現金部位有一一七○億美元，財務體質相當健全。霸榮周刊認為，一九年蘋果股價上漲超過八成，Alphabet一九年的營收每季以兩成的方式成長，但蘋果一九年的營收卻下滑。Alphabet旗下有Youtube、Android、Chrom、Google地圖和Waze等，Google則是全世界最頂尖的搜尋引擎。RBC Markets分析師Mark Makaney認為，Alphabet的財務體質很強壯，給予目標價一五○○美元。
霸榮周刊指出，如果Alphabet能向投資人釋出些善意訊息，例如增加買回庫藏股、分配現金股利、抑制費用成長和提供主要業務

----------------------------------------------------------------------

[74] 郭台銘對罹難者致上最崇高敬意　盼政府徹查黑鷹失事原因

▲鴻海創辦人郭台銘。（圖／記者林敬旻攝）
記者姚惠茹／台北報導
鴻海（2317）創辦人郭台銘今（2）日親自出席「第三屆鴻海獎學鯨」頒獎典禮，會後受訪被問到對於黑鷹事故的看法，郭台銘首先為罹難者與家屬致上最崇高的敬意，並強調希望政府可以徹查黑鷹失事原因，因為這已經是台灣第二次發生直升機意外，希望經過查證可以防止事故再次發生。
國軍一架黑鷹直升機迫降烏來山區，現場救出5名生還者，但仍造成8人罹難，其中包括參謀總長沈一鳴，對此，郭台銘今日出席活動受訪表示，這是非常重大的國安事件， 尤其是參謀總長不幸因公殉職罹難，這是國家安全非常重要的職位，希望政府提高警備，盡快做好人事調整工作。
郭台銘首先為罹難者與家屬致上最崇高的敬意，並強調對於罹難者，政府除了應該給他們撫卹之外，更希望可以徹查事故發生原因，尤其這是台灣第二次發生直升機意外事故，希望藉由查證以防止未來不再發生同樣的憾事。
隨後有記者問出政治議題，郭台銘則回應，今日這個嚴肅的場合不要談政治，他只是在閒暇時間希望做有意義的事情，初心不是要從事政治活動，只是發現台灣有太多需要被幫助的人，而他是一位實業家，所以不會只說而是去做，像是鴻海獎學金這三年幫助到很多人，而他只是提供一點點風力，讓學生可以展翅高飛。
----------------------------------------------------------------------

[75] 失能險理賠關鍵曝光！「一次金+扶助金」照護生活　專家解析：可優先把一次金額提高

▲失能後無法工作，醫療費用與看護費用是龐大的負擔，民眾須注意失能險的重要性。（圖／pakutaso）
記者楊絡懸／台北報導
失能險商品去年熱賣，鑑於金管會政策影響，保險公司也調整失能險的投保規則與商品內容，今年也將面臨保費調漲的可能。當失能險內容改變後，保戶申請理賠時需要注意什麼情況呢？錠嵂保險經紀人整理重點，解析失能險的理賠關鍵。
不過，錠嵂保經提醒注意2重點：如果失能扶助金是「月給付」，保單生效後啟動理賠給付，每月給付一筆扶助金；如果失能扶助金是「年給付」，保單生效後，必須等到隔年的

記者姚惠茹／台北報導
保瑞藥業（6472）今（2）日宣布，旗下適用於帕金森氏症的長效型療效膠囊(Numient)取得衛福部藥品查驗登記許可後，已於去年底將首批產品陸續佈建到各通路，目前以自費用藥為主，未來也將爭取健保用藥給付，以產品佈點和代工狀況來看，可望成為保瑞今年的營運動能。
保瑞表示，前年併購益邦時一同取得Numient的權利，保瑞接手益邦後，不但持續幫原廠代工，外銷美國，並以新劑型、新使用劑量、新單位含量新藥向衛福部申請獲得許可，不僅在台銷售，而且隨著人口老化問題嚴重，目前全球有800萬人受帕金森氏症所苦，因此每年的代工需求也持續擴大。
保瑞指出，以現行醫療技術而言，帕金森氏症是一種不可逆的疾病，一旦罹患就只能透過藥物減緩退化，也因此相關藥物的需求逐年增加。依據研究調查機構Market Data Forecast指出，2018年治療帕金森氏症藥物光是在亞洲每年就有10.4億美元的市場，預估年複合增長率約6.8％。
保瑞說明，帕金森氏症是一種「運動功能」的神經變性疾病，患者可能因隨著年齡增長，腦內的多巴胺神經細胞數量逐漸減少，並隨著耗損達一定程度就會產生相關症狀，而Numient是屬於一線用藥，原廠預估該款產品全球市場規模約可達台幣100億元。
法人預估，保瑞帕金森氏症的長效型療效膠囊在台上市後，目前已經將首批產品陸續佈建到各通路，未來也有機會在全台每年10億台幣的帕金森氏症市場中佔有一席之地，成為保瑞後續重要的營運動能之一。
----------------------------------------------------------------------

[87] 大量解僱人數創單月新高　勞動部積極推動入廠查訪機制

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
勞動部今(2)日公布去（108）年11月大量解僱統計件數及人數，1至11月底大量解僱總件數為189件，遭解僱人數為1萬5,045人。遭解僱人數為近7年史上最高，但勞動部指出，11月單月計畫解僱人數雖較歷年高，主因為「中華映管」在去年11月30日解僱1900人所致。
勞動部進一步說明，去（108）年迄至11月底，事業單位通報大量解僱總件數計189件，為自102年至今之新低，顯示我國整體企業經營情況較往年相對穩定。又1到11月計畫解僱總人數計15,045人，以及11月單月計畫解僱人數